# Stacked Bar Graph
## Tech Stack Count by Role

In [26]:
import pandas
from altair import Chart, Tooltip, Color, X, Y
from pandas import DataFrame

from app.data import MongoDB

### Data Source

In [27]:
db = MongoDB()

### Data Engineering

In [28]:
mentees = DataFrame(
    db.projection("Mentees", {"is_active": True, "validate_status": "approved"},
                  {"tech_stack": True}))
mentees

,tech_stack
0,Data Science
1,Career Development
2,Frontend
3,Data Science
4,iOS
...,...
97,string
98,string
99,string
100,string


In [29]:
mentees["role"] = "Mentee"
mentees

,tech_stack,role
0,Data Science,Mentee
1,Career Development,Mentee
2,Frontend,Mentee
3,Data Science,Mentee
4,iOS,Mentee
...,...,...
97,string,Mentee
98,string,Mentee
99,string,Mentee
100,string,Mentee


In [30]:
mentors = DataFrame(db.projection("Mentors", {
    "accepting_new_mentees": True,
    "is_active": True,
    "validate_status": "approved",
}, {"tech_stack": True}))
mentors = mentors.explode(column="tech_stack")
mentors["role"] = "Mentor"
mentors


,tech_stack,role
0,Backend,Mentor
0,Android,Mentor
1,iOS,Mentor
2,Backend,Mentor
2,iOS,Mentor
3,Frontend,Mentor
3,iOS,Mentor
3,Backend,Mentor
4,Backend,Mentor
4,iOS,Mentor


In [31]:
df = pandas.concat([mentees, mentors])
df.sample(10)

,tech_stack,role
58,Design UI/UX,Mentee
15,iOS,Mentee
63,Career Development,Mentee
6,Data Science,Mentee
27,Android,Mentee
28,iOS,Mentee
41,Design UI/UX,Mentee
13,Career Development,Mentee
12,Frontend,Mentor
17,Backend,Mentor


### Data Visualization


In [32]:
def title_fix(string: str) -> str:
    return string.title().replace("_", " ")

In [33]:
def stacked_bar_chart(df, column_1, column_2):
    return Chart(
        df,
        title=f"{title_fix(column_1)} Count by {title_fix(column_2)}",
    ).mark_bar().encode(
        x=X(column_1, title=title_fix(column_1), sort="-y"),
        y=Y(f"count({column_1})"),
        color=Color(column_2, title=title_fix(column_2)),
        tooltip=Tooltip([column_2, column_1, f"count({column_1})"])
    ).properties(
        width=480,
        height=400,
        padding=24,
    ).configure(
        legend={"padding": 24},
        title={"fontSize": 20, "offset": 24},
        view={"stroke": "#FFF"},
    )



In [34]:
chart = stacked_bar_chart(df, "tech_stack", "role")
chart

alt.Chart(...)

In [35]:
chart.to_dict()


{'config': {'view': {'continuousWidth': 400,
   'continuousHeight': 300,
   'stroke': '#FFF'},
  'legend': {'padding': 24},
  'title': {'fontSize': 20, 'offset': 24}},
 'data': {'name': 'data-ac832bdbf98d38c0ff7a097466b742e5'},
 'mark': 'bar',
 'encoding': {'color': {'field': 'role', 'title': 'Role', 'type': 'nominal'},
  'tooltip': [{'field': 'role', 'type': 'nominal'},
   {'field': 'tech_stack', 'type': 'nominal'},
   {'aggregate': 'count', 'field': 'tech_stack', 'type': 'nominal'}],
  'x': {'field': 'tech_stack',
   'sort': '-y',
   'title': 'Tech Stack',
   'type': 'nominal'},
  'y': {'aggregate': 'count', 'field': 'tech_stack', 'type': 'nominal'}},
 'height': 400,
 'padding': 24,
 'title': 'Tech Stack Count by Role',
 'width': 480,
 '$schema': 'https://vega.github.io/schema/vega-lite/v4.17.0.json',
 'datasets': {'data-ac832bdbf98d38c0ff7a097466b742e5': [{'tech_stack': 'Data Science',
    'role': 'Mentee'},
   {'tech_stack': 'Career Development', 'role': 'Mentee'},
   {'tech_stack'

In [39]:
import requests

chart = requests.get("http://127.0.0.1:8000/graph?column1=tech_stack&column2=role").json()
Chart.from_dict(chart)


alt.Chart(...)